In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
from pathlib import Path
from multiprocessing.dummy import Pool as ThreadPool
from collections import defaultdict
from natsort import natsorted
import tsfresh as tf
import sklearn

In [3]:
pd.set_option('max_columns', None)

### Data loading

In [4]:
import os
import glob

dataset_path = 'data_feats_90_c/'
csv_files = glob.glob(dataset_path+'*.csv')

all_df = []

for filename in csv_files:
    df = pd.read_csv(filename, index_col='Unnamed: 0', header=0)
    all_df.append(df)

df = pd.concat(all_df, axis=0, ignore_index=False)
df.sort_index()

,P-PDG__mean,P-PDG__variance,P-PDG__skewness,P-PDG__kurtosis,"P-PDG__fft_aggregated__aggtype_""centroid""","P-PDG__fft_aggregated__aggtype_""variance""","P-PDG__fft_aggregated__aggtype_""skew""","P-PDG__fft_aggregated__aggtype_""kurtosis""",P-PDG__maximum,P-PDG__minimum,P-PDG__median,P-PDG__quantile__q_0.1,P-PDG__quantile__q_0.2,P-PDG__quantile__q_0.3,P-PDG__quantile__q_0.4,P-PDG__quantile__q_0.6,P-PDG__quantile__q_0.7,P-PDG__quantile__q_0.8,P-PDG__quantile__q_0.9,P-PDG__variation_coefficient,P-PDG__mean_change,P-PDG__mean_second_derivative_central,P-PDG__friedrich_coefficients__coeff_1__m_3__r_30,P-PDG__friedrich_coefficients__coeff_3__m_3__r_30,P-TPT__mean,P-TPT__variance,P-TPT__skewness,P-TPT__kurtosis,"P-TPT__fft_aggregated__aggtype_""centroid""","P-TPT__fft_aggregated__aggtype_""variance""","P-TPT__fft_aggregated__aggtype_""skew""","P-TPT__fft_aggregated__aggtype_""kurtosis""",P-TPT__maximum,P-TPT__minimum,P-TPT__median,P-TPT__quantile__q_0.1,P-TPT__quantile__q_0.2,P-TPT__quantile__q_0.3,P-TPT__quantile__q_0.4,P-TPT__quantile__q_0.6,P-TPT__quantile__q_0.7,P-TPT__quantile__q_0.8,P-TPT__quantile__q_0.9,P-TPT__variation_coefficient,P-TPT__mean_change,P-TPT__mean_second_derivative_central,P-TPT__friedrich_coefficients__coeff_1__m_3__r_30,P-TPT__friedrich_coefficients__coeff_3__m_3__r_30,T-TPT__mean,T-TPT__variance,T-TPT__skewness,T-TPT__kurtosis,"T-TPT__fft_aggregated__aggtype_""centroid""","T-TPT__fft_aggregated__aggtype_""variance""","T-TPT__fft_aggregated__aggtype_""skew""","T-TPT__fft_aggregated__aggtype_""kurtosis""",T-TPT__maximum,T-TPT__minimum,T-TPT__median,T-TPT__quantile__q_0.1,T-TPT__quantile__q_0.2,T-TPT__quantile__q_0.3,T-TPT__quantile__q_0.4,T-TPT__quantile__q_0.6,T-TPT__quantile__q_0.7,T-TPT__quantile__q_0.8,T-TPT__quantile__q_0.9,T-TPT__variation_coefficient,T-TPT__mean_change,T-TPT__mean_second_derivative_central,T-TPT__friedrich_coefficients__coeff_1__m_3__r_30,T-TPT__friedrich_coefficients__coeff_3__m_3__r_30,P-MON-CKP__mean,P-MON-CKP__variance,P-MON-CKP__skewness,P-MON-CKP__kurtosis,"P-MON-CKP__fft_aggregated__aggtype_""centroid""","P-MON-CKP__fft_aggregated__aggtype_""variance""","P-MON-CKP__fft_aggregated__aggtype_""skew""","P-MON-CKP__fft_aggregated__aggtype_""kurtosis""",P-MON-CKP__maximum,P-MON-CKP__minimum,P-MON-CKP__median,P-MON-CKP__quantile__q_0.1,P-MON-CKP__quantile__q_0.2,P-MON-CKP__quantile__q_0.3,P-MON-CKP__quantile__q_0.4,P-MON-CKP__quantile__q_0.6,P-MON-CKP__quantile__q_0.7,P-MON-CKP__quantile__q_0.8,P-MON-CKP__quantile__q_0.9,P-MON-CKP__variation_coefficient,P-MON-CKP__mean_change,P-MON-CKP__mean_second_derivative_central,P-MON-CKP__friedrich_coefficients__coeff_1__m_3__r_30,P-MON-CKP__friedrich_coefficients__coeff_3__m_3__r_30,T-JUS-CKP__mean,T-JUS-CKP__variance,T-JUS-CKP__skewness,T-JUS-CKP__kurtosis,"T-JUS-CKP__fft_aggregated__aggtype_""centroid""","T-JUS-CKP__fft_aggregated__aggtype_""variance""","T-JUS-CKP__fft_aggregated__aggtype_""skew""","T-JUS-CKP__fft_aggregated__aggtype_""kurtosis""",T-JUS-CKP__maximum,T-JUS-CKP__minimum,T-JUS-CKP__median,T-JUS-CKP__quantile__q_0.1,T-JUS-CKP__quantile__q_0.2,T-JUS-CKP__quantile__q_0.3,T-JUS-CKP__quantile__q_0.4,T-JUS-CKP__quantile__q_0.6,T-JUS-CKP__quantile__q_0.7,T-JUS-CKP__quantile__q_0.8,T-JUS-CKP__quantile__q_0.9,T-JUS-CKP__variation_coefficient,T-JUS-CKP__mean_change,T-JUS-CKP__mean_second_derivative_central,T-JUS-CKP__friedrich_coefficients__coeff_1__m_3__r_30,T-JUS-CKP__friedrich_coefficients__coeff_3__m_3__r_30,QGL__mean,QGL__variance,QGL__skewness,QGL__kurtosis,"QGL__fft_aggregated__aggtype_""centroid""","QGL__fft_aggregated__aggtype_""variance""","QGL__fft_aggregated__aggtype_""skew""","QGL__fft_aggregated__aggtype_""kurtosis""",QGL__maximum,QGL__minimum,QGL__median,QGL__quantile__q_0.1,QGL__quantile__q_0.2,QGL__quantile__q_0.3,QGL__quantile__q_0.4,QGL__quantile__q_0.6,QGL__quantile__q_0.7,QGL__quantile__q_0.8,QGL__quantile__q_0.9,QGL__variation_coefficient,QGL__mean_change,QGL__mean_second_derivative_central,QGL__friedrich_coefficients

In [5]:
#df.drop(['Unnamed: 0'], axis='columns', inplace=True)      #leftover index column, unnecessary

df = df[df.class_code != 7]                                #paper does not use event class 7
df['class_code'].replace({8: 7}, inplace=True)


### Handling NA values

In [6]:
def print_na_sum(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        nans = df.isna().sum(axis=0)
        print(nans[nans!=0])

print_na_sum(df)

P-PDG__fft_aggregated__aggtype_"centroid"                 8571
P-PDG__fft_aggregated__aggtype_"variance"                 8571
P-PDG__fft_aggregated__aggtype_"skew"                    37244
P-PDG__fft_aggregated__aggtype_"kurtosis"                37244
P-PDG__variation_coefficient                              8571
P-PDG__friedrich_coefficients__coeff_1__m_3__r_30        20675
P-PDG__friedrich_coefficients__coeff_3__m_3__r_30        20675
P-TPT__fft_aggregated__aggtype_"centroid"                   64
P-TPT__fft_aggregated__aggtype_"variance"                   64
P-TPT__fft_aggregated__aggtype_"skew"                    30450
P-TPT__fft_aggregated__aggtype_"kurtosis"                30450
P-TPT__variation_coefficient                                64
P-TPT__friedrich_coefficients__coeff_1__m_3__r_30        12623
P-TPT__friedrich_coefficients__coeff_3__m_3__r_30        12623
T-TPT__fft_aggregated__aggtype_"centroid"                 6514
T-TPT__fft_aggregated__aggtype_"variance"              


Removing features with too many NA values

In [7]:
#df.drop(['QGL__variation_coefficient'], axis='columns', inplace=True)
df.drop(list(df.filter(regex = 'friedrich')), axis = 1, inplace = True)
df.drop(list(df.filter(regex = 'fft_aggregated')), axis = 1, inplace = True)

In [8]:
print_na_sum(df)

P-PDG__variation_coefficient         8571
P-TPT__variation_coefficient           64
T-TPT__variation_coefficient         6514
P-MON-CKP__variation_coefficient     1200
T-JUS-CKP__variation_coefficient     1819
QGL__variation_coefficient          50288
dtype: int64


Imputing leftover na values, for each class separately

In [9]:
imputed_df_list = []
for i in df['class_code'].unique():
    query = 'class_code == ' + str(i)
    imputed = tf.utilities.dataframe_functions.impute(df.query(query))
    imputed_df_list.append(imputed)
imputed_df = pd.concat(imputed_df_list)

/home/marko/miniconda3/lib/python3.8/site-packages/pandas/core/generic.py:9032: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(cond, other, inplace, axis, level, errors=errors)
/home/marko/miniconda3/lib/python3.8/site-packages/tsfresh/utilities/dataframe_functions.py:198: RuntimeWarning: The columns ['QGL__variation_coefficient'] did not have any finite values. Filling with zeros.
  warnings.warn(


In [10]:
print_na_sum(imputed_df)

Series([], dtype: int64)


In [11]:
imputed_df = imputed_df.sample(frac=1).reset_index(drop=True)
sample_df = imputed_df#.sample(frac=0.01, random_state=42)
X = sample_df.iloc[:,:-1].to_numpy()
y = sample_df.iloc[:,-1].to_numpy()
sample_df.shape

(51764, 109)

In [11]:
X

array([[ 3.36066649e+07,  2.51276744e+04,  1.58291762e+00, ...,
         0.00000000e+00, -7.40663470e+01, -8.39132535e+01],
       [ 2.55759415e+07,  3.40720383e+05,  6.65568965e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 4.48580500e+07,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  4.60261588e+01,  8.45119623e+01],
       ...,
       [ 2.44106438e+07,  3.64050855e+09, -4.78281533e-01, ...,
         0.00000000e+00,  1.17108400e+02,  1.07422217e+02],
       [ 2.71790223e+07,  2.35244108e+03, -8.37967863e-02, ...,
         0.00000000e+00, -8.57896789e+01, -8.65998877e+01],
       [ 1.92935724e+07,  4.92172840e+00,  1.65714215e-02, ...,
         0.00000000e+00, -8.54150679e+01, -8.61854463e+01]])

In [12]:
y

array([7, 0, 0, ..., 3, 5, 6])

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
from sklearn.model_selection import KFold   

#todo: include window size into grid search

window_size_hp = [300, 600, 900]
regularization_hp = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1] #p.logspace

In [15]:
from sklearn.model_selection import KFold   
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

kf = KFold(n_splits=5)

### Logistic regression, SVM

In [16]:
from sklearn.linear_model import SGDClassifier

best_logreg_score = 0
best_logreg = None

for clf in ['log', 'hinge']:
    if clf == 'log': 
        print('Logistic regression')
    if clf == 'hinge':
        print('Support vector machine')
    for r in regularization_hp:
        i = 0
        for train_index, test_index in kf.split(X_train):
            X_k_train, X_k_test = X_train[train_index], X_train[test_index]
            y_k_train, y_k_test = y_train[train_index], y_train[test_index]
            scaler = StandardScaler().fit(X_k_train)
            X_k_train = scaler.transform(X_k_train)
            X_k_test = scaler.transform(X_k_test)
            log_reg = SGDClassifier(loss='log', max_iter=10000, alpha=r).fit(X_k_train, y_k_train)
            h = log_reg.predict(X_k_test)
            X_test_scaled = scaler.transform(X_test)
            h_test = log_reg.predict(X_test_scaled)
            print('r='+str(r)+', i-th fold='+str(i)+', k-fold f1 score='+str(f1_score(y_k_test, h, average='macro'))+
                 ' , test f1 score=' + str(f1_score(y_test, h_test, average='macro')))
            i += 1
            
            if f1_score(y_test, h_test, average='macro') > best_logreg_score:
                best_logreg_score = f1_score(y_test, h_test, average='macro')
                best_logreg = log_reg
            
    print(confusion_matrix(y_test, h_test, normalize='true'))
    print('best_score', best_logreg_score)
    

Logistic regression
r=1e-07, i-th fold=0, k-fold f1 score=0.8934146416774478 , test f1 score=0.8938743814997794
r=1e-07, i-th fold=1, k-fold f1 score=0.880134487473437 , test f1 score=0.8670098164258693
r=1e-07, i-th fold=2, k-fold f1 score=0.8656129247201896 , test f1 score=0.8757347580440408
r=1e-07, i-th fold=3, k-fold f1 score=0.90448216234355 , test f1 score=0.9094595665854834
r=1e-07, i-th fold=4, k-fold f1 score=0.9114415891803932 , test f1 score=0.9140093073278454
r=1e-06, i-th fold=0, k-fold f1 score=0.8865390818068251 , test f1 score=0.8943567866197979
r=1e-06, i-th fold=1, k-fold f1 score=0.890210385788267 , test f1 score=0.8935443574623818
r=1e-06, i-th fold=2, k-fold f1 score=0.8922511267917972 , test f1 score=0.9120259010566922
r=1e-06, i-th fold=3, k-fold f1 score=0.89301228754006 , test f1 score=0.8944306935132752
r=1e-06, i-th fold=4, k-fold f1 score=0.9001304287645278 , test f1 score=0.8962926961521953
r=1e-05, i-th fold=0, k-fold f1 score=0.909930711384981 , test f1 

### Decision trees

In [17]:
from sklearn.tree import DecisionTreeClassifier


best_tree_score = 0
best_tree = None


complexity_hp = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
for ccp in complexity_hp:
    i = 0
    for train_index, test_index in kf.split(X_train):
        X_k_train, X_k_test = X_train[train_index], X_train[test_index]
        y_k_train, y_k_test = y_train[train_index], y_train[test_index]
        scaler = StandardScaler().fit(X_k_train)
        X_k_train = scaler.transform(X_k_train)
        X_k_test = scaler.transform(X_k_test)
        tree = DecisionTreeClassifier(random_state=42, ccp_alpha=ccp).fit(X_k_train, y_k_train)
        h = tree.predict(X_k_test)
        X_test_scaled = scaler.transform(X_test)
        h_test = tree.predict(X_test_scaled)
        print('ccp='+str(ccp)+', i-th fold='+str(i)+', k-fold f1 score='+str(f1_score(y_k_test, h, average='macro'))+
            ' , test f1 score=' + str(f1_score(y_test, h_test, average='macro')))
        i += 1
        
        if f1_score(y_test, h_test, average='macro') > best_tree_score:
                best_tree_score = f1_score(y_test, h_test, average='macro')
                best_tree = tree
                
confusion_matrix(y_test, h_test, normalize='true')
print('best_score', best_tree_score)

ccp=0.1, i-th fold=0, k-fold f1 score=0.41435932845579426 , test f1 score=0.420588116151728
ccp=0.1, i-th fold=1, k-fold f1 score=0.42105634172308487 , test f1 score=0.4195550301638137
ccp=0.1, i-th fold=2, k-fold f1 score=0.4202302906134777 , test f1 score=0.4203985286655612
ccp=0.1, i-th fold=3, k-fold f1 score=0.4134527090153043 , test f1 score=0.4194986309024872
ccp=0.1, i-th fold=4, k-fold f1 score=0.42612760650588866 , test f1 score=0.427462325259361
ccp=0.01, i-th fold=0, k-fold f1 score=0.9192896912400812 , test f1 score=0.9168496675801787
ccp=0.01, i-th fold=1, k-fold f1 score=0.7941022503299885 , test f1 score=0.8045390449892388
ccp=0.01, i-th fold=2, k-fold f1 score=0.8128070633541434 , test f1 score=0.8227554569246383
ccp=0.01, i-th fold=3, k-fold f1 score=0.9134907038431448 , test f1 score=0.9141773912483997
ccp=0.01, i-th fold=4, k-fold f1 score=0.9024821646530351 , test f1 score=0.9041666161485278
ccp=0.001, i-th fold=0, k-fold f1 score=0.9639649684772982 , test f1 score

### Random forest

In [18]:
from sklearn.ensemble import RandomForestClassifier


best_trees_score = 0
best_trees = None
        
n_trees_hp = [50, 100, 150, 175]
max_depth_hp = [5, 7, 10, None]
n_feats_at_splits_hp = [5, 10, 15]   #pitati za komentar o ovom hiperparametru

for n_trees in n_trees_hp:
    for max_depth in max_depth_hp:     #iterations model, product za nested for petlje itertools
        for n_feats_at_splits in n_feats_at_splits_hp:
            i = 0
            for train_index, test_index in kf.split(X_train):
                X_k_train, X_k_test = X_train[train_index], X_train[test_index]
                y_k_train, y_k_test = y_train[train_index], y_train[test_index]
                scaler = StandardScaler().fit(X_k_train)
                X_k_train = scaler.transform(X_k_train)
                X_k_test = scaler.transform(X_k_test)
                trees = RandomForestClassifier(random_state=42, n_estimators=n_trees,
                                              max_depth=max_depth, min_samples_split=n_feats_at_splits)
                trees.fit(X_k_train, y_k_train)
                h = trees.predict(X_k_test)
                X_test_scaled = scaler.transform(X_test)
                h_test = trees.predict(X_test_scaled)
                print('n_feats_at_split=' + str(n_feats_at_splits) +', n_trees=' + str(n_trees) + 
                'max depth:'+ str(max_depth) + ', i-th fold='+str(i)+', k-fold f1 score='+
                str(f1_score(y_k_test, h, average='macro'))+
                ' , test f1 score=',str(f1_score(y_test, h_test, average='macro')))
                i += 1
                
                if f1_score(y_test, h_test, average='macro') > best_trees_score:
                    best_trees_score = f1_score(y_test, h_test, average='macro')
                    best_trees = trees
                
confusion_matrix(y_test, h_test, normalize='true')
print('best_score', best_trees_score)

n_feats_at_split=5, n_trees=50max depth:5, i-th fold=0, k-fold f1 score=0.832471382716526 , test f1 score= 0.8379813488056549
n_feats_at_split=5, n_trees=50max depth:5, i-th fold=1, k-fold f1 score=0.847051542021106 , test f1 score= 0.8425125333636097
n_feats_at_split=5, n_trees=50max depth:5, i-th fold=2, k-fold f1 score=0.8294054400863448 , test f1 score= 0.8440677618912729
n_feats_at_split=5, n_trees=50max depth:5, i-th fold=3, k-fold f1 score=0.9089981574547545 , test f1 score= 0.9000072344231703
n_feats_at_split=5, n_trees=50max depth:5, i-th fold=4, k-fold f1 score=0.8199182945630625 , test f1 score= 0.8095968593206382
n_feats_at_split=10, n_trees=50max depth:5, i-th fold=0, k-fold f1 score=0.908536492352539 , test f1 score= 0.9007401437219291
n_feats_at_split=10, n_trees=50max depth:5, i-th fold=1, k-fold f1 score=0.883709884441463 , test f1 score= 0.8865257331165151
n_feats_at_split=10, n_trees=50max depth:5, i-th fold=2, k-fold f1 score=0.8270572989155063 , test f1 score= 0.84

KeyboardInterrupt: 

### Linear Discriminant Analysis, Quadratic Discriminant Analysis

In [19]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

best_lda_score = 0
best_lda = None

print('LDA')
i = 0
for train_index, test_index in kf.split(X_train):
    X_k_train, X_k_test = X_train[train_index], X_train[test_index]
    y_k_train, y_k_test = y_train[train_index], y_train[test_index]
    scaler = StandardScaler().fit(X_k_train)
    X_k_train = scaler.transform(X_k_train)
    X_k_test = scaler.transform(X_k_test)
    clf = LinearDiscriminantAnalysis().fit(X_k_train, y_k_train)
    h = clf.predict(X_k_test)
    X_test_scaled = scaler.transform(X_test)
    h_test = clf.predict(X_test_scaled)
    print('i-th fold='+str(i)+', k-fold f1 score='+str(f1_score(y_k_test, h, average='macro'))+
         ' , test f1 score=' + str(f1_score(y_test, h_test, average='macro')))
    i += 1
    
    if f1_score(y_test, h_test, average='macro') > best_lda_score:
                best_lda_score = f1_score(y_test, h_test, average='macro')
                best_lda = clf
                
print(confusion_matrix(y_test, h_test, normalize='true'))
print('best_score', best_lda_score)


best_qda_score = 0
best_qda = None

print('\nQDA')
i = 0
for train_index, test_index in kf.split(X_train):
    X_k_train, X_k_test = X_train[train_index], X_train[test_index]
    y_k_train, y_k_test = y_train[train_index], y_train[test_index]
    scaler = StandardScaler().fit(X_k_train)
    X_k_train = scaler.transform(X_k_train)
    X_k_test = scaler.transform(X_k_test)
    clf = QuadraticDiscriminantAnalysis().fit(X_k_train, y_k_train)
    h = clf.predict(X_k_test)
    X_test_scaled = scaler.transform(X_test)
    h_test = clf.predict(X_test_scaled)
    print('i-th fold='+str(i)+', k-fold f1 score='+str(f1_score(y_k_test, h, average='macro'))+
         ' , test f1 score=' + str(f1_score(y_test, h_test, average='macro')))
    i += 1
    
    if f1_score(y_test, h_test, average='macro') > best_qda_score:
                best_qda_score = f1_score(y_test, h_test, average='macro')
                best_qda = clf
                
    
confusion_matrix(y_test, h_test, normalize='true')
print('best_score', best_qda_score)

LDA
i-th fold=0, k-fold f1 score=0.8495107770177578 , test f1 score=0.8543661120056427
i-th fold=1, k-fold f1 score=0.8735420380266964 , test f1 score=0.8525625679185176
i-th fold=2, k-fold f1 score=0.8534120595900301 , test f1 score=0.8554097438083555
i-th fold=3, k-fold f1 score=0.8479786226196453 , test f1 score=0.8542902566117151
i-th fold=4, k-fold f1 score=0.8577982007615702 , test f1 score=0.8543140118853287
[[9.90430622e-01 1.36705400e-03 0.00000000e+00 0.00000000e+00
  4.10116200e-03 3.41763500e-03 6.83526999e-04 0.00000000e+00]
 [8.46859562e-03 8.45448130e-01 0.00000000e+00 0.00000000e+00
  7.05716302e-04 1.44671842e-01 7.05716302e-04 0.00000000e+00]
 [4.25531915e-02 2.12765957e-02 7.76595745e-01 2.12765957e-02
  2.12765957e-02 9.57446809e-02 1.06382979e-02 1.06382979e-02]
 [2.54777070e-03 5.47770701e-02 5.09554140e-03 8.08917197e-01
  1.27388535e-03 1.27388535e-01 0.00000000e+00 0.00000000e+00]
 [1.18181818e-01 1.00000000e-01 0.00000000e+00 0.00000000e+00
  7.63636364e-01 9.

/home/marko/miniconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


i-th fold=0, k-fold f1 score=0.6671132525485376 , test f1 score=0.6714184991836034


/home/marko/miniconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


i-th fold=1, k-fold f1 score=0.8061185861655584 , test f1 score=0.8154633434506278


...

i-th fold=2, k-fold f1 score=0.6847773657862855 , test f1 score=0.6796514655447179


/home/marko/miniconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


i-th fold=3, k-fold f1 score=0.8377553149476118 , test f1 score=0.8466906595006162


/home/marko/miniconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


i-th fold=4, k-fold f1 score=0.7971017664271912 , test f1 score=0.8071700016718247
best_score 0.8466906595006162


In [20]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

best_ada_score = 0
best_ada = None

n_trees_hp = [250, 400, 100, 550]
max_depth_hp = [3, 1, 5]
lr_hp = [0.1] #001 01 1

for max_depth in max_depth_hp:
    for n_trees in n_trees_hp:
        for lr in lr_hp:
            i = 0
            for train_index, test_index in kf.split(X_train):
                X_k_train, X_k_test = X_train[train_index], X_train[test_index]
                y_k_train, y_k_test = y_train[train_index], y_train[test_index]
                scaler = StandardScaler().fit(X_k_train)
                X_k_train = scaler.transform(X_k_train)
                X_k_test = scaler.transform(X_k_test)
                ada = AdaBoostClassifier(random_state=42, n_estimators=n_trees,
                                            base_estimator=DecisionTreeClassifier(max_depth=max_depth))
                ada.fit(X_k_train, y_k_train)
                h = ada.predict(X_k_test)
                X_test_scaled = scaler.transform(X_test)
                h_test = ada.predict(X_test_scaled)
                print('n_trees=' + str(n_trees) +', max_depth=' + str(max_depth) + 
                      ', lr=' + str(lr) + ', i-th fold='+str(i)+', k-fold f1 score='+
                      str(f1_score(y_k_test, h, average='macro'))+
                    ' , test f1 score=',str(f1_score(y_test, h_test, average='macro')))
                i += 1
                
                if f1_score(y_test, h_test, average='macro') > best_ada_score:
                    best_ada_score = f1_score(y_test, h_test, average='macro')
                    best_ada = ada

    
confusion_matrix(y_test, h_test, normalize='true')
print('best_score', best_ada_score)

n_trees=250, max_depth=3, lr=0.1, i-th fold=0, k-fold f1 score=0.961653083237205 , test f1 score= 0.9641957966060913


KeyboardInterrupt: 

In [ ]:
n_trees=100, max_depth=3, lr=0.01, i-th fold=0, k-fold f1 score=0.9098299746638029 , test f1 score= 0.9090440173723331
n_trees=100, max_depth=3, lr=0.01, i-th fold=1, k-fold f1 score=0.9200426525551122 , test f1 score= 0.9158788102994806
n_trees=100, max_depth=3, lr=0.01, i-th fold=2, k-fold f1 score=0.9509625266945874 , test f1 score= 0.9491826272735017
n_trees=100, max_depth=3, lr=0.01, i-th fold=3, k-fold f1 score=0.9285494129988294 , test f1 score= 0.9252001147201362
n_trees=100, max_depth=3, lr=0.01, i-th fold=4, k-fold f1 score=0.8813602357765438 , test f1 score= 0.8791765773337183
n_trees=100, max_depth=3, lr=0.1, i-th fold=0, k-fold f1 score=0.9098299746638029 , test f1 score= 0.9090440173723331
n_trees=100, max_depth=3, lr=0.1, i-th fold=1, k-fold f1 score=0.9200426525551122 , test f1 score= 0.9158788102994806

In [21]:
from sklearn.neighbors import KNeighborsClassifier

best_knn_score = 0
best_knn = None

n_neighbours_hp = [5, 10, 30, 50]

for n_neigh in n_neighbours_hp:
    i = 0
    for train_index, test_index in kf.split(X_train):
        X_k_train, X_k_test = X_train[train_index], X_train[test_index]
        y_k_train, y_k_test = y_train[train_index], y_train[test_index]
        scaler = StandardScaler().fit(X_k_train)
        X_k_train = scaler.transform(X_k_train)
        X_k_test = scaler.transform(X_k_test)
        
        
        knn = KNeighborsClassifier(n_neighbors=n_neigh).fit(X_k_train, y_k_train)
        h = knn.predict(X_k_test)
        X_test_scaled = scaler.transform(X_test)
        h_test = knn.predict(X_test_scaled)
        print('neighs='+str(n_neigh)+', k-fold f1 score='+str(f1_score(y_k_test, h, average='macro'))+
            ' , test f1 score=' + str(f1_score(y_test, h_test, average='macro')))
        i += 1

        if f1_score(y_test, h_test, average='macro') > best_knn_score:
            best_knn_score = f1_score(y_test, h_test, average='macro')
            best_knn = knn

confusion_matrix(y_test, h_test, normalize='true')
print('best_score', best_knn_score)

neighs=5, k-fold f1 score=0.9414280488494129 , test f1 score=0.9420099406672514
neighs=5, k-fold f1 score=0.9506039837125423 , test f1 score=0.9465326198175181
neighs=5, k-fold f1 score=0.9419149936480191 , test f1 score=0.9466225385712372
neighs=5, k-fold f1 score=0.9404771795720348 , test f1 score=0.9450233662061007
neighs=5, k-fold f1 score=0.9351109325076237 , test f1 score=0.943069036025647
neighs=10, k-fold f1 score=0.9263048128999589 , test f1 score=0.9345885418285385
neighs=10, k-fold f1 score=0.9425470698995871 , test f1 score=0.9366594155749253
neighs=10, k-fold f1 score=0.9263028252650342 , test f1 score=0.934620557289428
neighs=10, k-fold f1 score=0.9304246624871512 , test f1 score=0.9331019192769922
neighs=10, k-fold f1 score=0.9306801770880724 , test f1 score=0.9370026586156357
neighs=30, k-fold f1 score=0.9010995083065496 , test f1 score=0.9055093428688261
neighs=30, k-fold f1 score=0.9160416457336361 , test f1 score=0.9059781529525857
neighs=30, k-fold f1 score=0.889598

In [22]:
from sklearn.ensemble import VotingClassifier

for voting in ['soft', 'hard']:
    for train_index, test_index in kf.split(X_train):
        X_k_train, X_k_test = X_train[train_index], X_train[test_index]
        y_k_train, y_k_test = y_train[train_index], y_train[test_index]
        scaler = StandardScaler().fit(X_k_train)
        X_k_train = scaler.transform(X_k_train)
        X_k_test = scaler.transform(X_k_test)


        eclf1 = VotingClassifier(estimators=[
             ('logreg', best_logreg), ('tree', best_tree), ('trees', best_trees), ('knn', best_knn)], voting=voting)
        eclf1 = eclf1.fit(X_k_train, y_k_train)
        h = eclf1.predict(X_k_test)

        X_test_scaled = scaler.transform(X_test)
        h_test = eclf1.predict(X_test_scaled)
        print('k-fold f1 score='+str(f1_score(y_k_test, h, average='macro'))+
            ' , test f1 score=' + str(f1_score(y_test, h_test, average='macro')))
        i += 1


confusion_matrix(y_test, h_test, normalize='true')


/home/marko/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:445: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))
/home/marko/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:445: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))


k-fold f1 score=0.9798330800616502 , test f1 score=0.9802522580935003


/home/marko/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:445: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))


k-fold f1 score=0.9828350426831882 , test f1 score=0.9833246244261877


/home/marko/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:445: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))
/home/marko/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:445: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))


...

/home/marko/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:445: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))
/home/marko/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:445: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))


k-fold f1 score=0.9783149180183828 , test f1 score=0.977833966690011


/home/marko/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:445: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))


k-fold f1 score=0.9896289138962797 , test f1 score=0.9854187354059909
k-fold f1 score=0.9794885492332559 , test f1 score=0.9769581346041882
k-fold f1 score=0.9774972505907895 , test f1 score=0.9727918664589549
k-fold f1 score=0.9762275339058332 , test f1 score=0.9751349272229586
k-fold f1 score=0.975356235152242 , test f1 score=0.9772311731665677
k-fold f1 score=0.979490682488693 , test f1 score=0.9800832314116964


array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.11714891e-03, 9.97882851e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.25531915e-02, 0.00000000e+00, 8.61702128e-01, 0.00000000e+00,
        0.00000000e+00, 5.31914894e-02, 0.00000000e+00, 4.25531915e-02],
       [0.00000000e+00, 0.00000000e+00, 2.54777070e-03, 9.89808917e-01,
        5.09554140e-03, 2.54777070e-03, 0.00000000e+00, 0.00000000e+00],
       [1.51515152e-02, 0.00000000e+00, 0.00000000e+00, 1.21212121e-02,
        9.72727273e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.27643785e-04, 4.63821892e-04, 4.63821892e-04, 6.49350649e-03,
        0.00000000e+00, 9.91651206e-01, 0.00000000e+00, 0.00000000e+00],
       [2.00000000e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 9.98000000e-01, 0.